In [33]:
import pandas as pd
train_pos=pd.read_csv('./data/imdb/train/pos.csv')
train_neg=pd.read_csv('./data/imdb/train/neg.csv')
test_pos=pd.read_csv('./data/imdb/test/pos.csv')
test_neg=pd.read_csv('./data/imdb/test/neg.csv')
print('imported data.....')

def clear_text(text):
    return text.replace('"', '').replace("'", "").replace("<br />","").replace(",","").replace("(","").replace(")","").replace("-","").replace("/","").replace("!","")
# train_pos['text']=train_pos['text'].apply(clear_text)
# print(train_pos[train_pos.id==0]['text'])
# for word in train_pos[train_pos.id==0]['text']:
#     print(word,end=' ')


# li=train_pos[train_pos.id==0]['text'].apply(lambda x:x.split('.'))
# for word in li:
#     print(word,end=' ')
# # print(train_pos['t'])

train_pos_sent=train_pos['text'].apply(clear_text).tolist()
train_neg_sent=train_neg['text'].apply(clear_text).tolist()
test_pos_sent=test_pos['text'].apply(clear_text).tolist()
test_neg_sent=test_neg['text'].apply(clear_text).tolist()


corpus=train_pos_sent+train_neg_sent+test_pos_sent+test_neg_sent


imported data.....


In [34]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
from nltk.corpus import stopwords
words=[]
for review in corpus:
    review_words=[]
    review_sents=review.split(".")
    for sent in review_sents:
        review_words.extend(sent.split(" "))
#         review_words.extend([word for word in sent.split(" ") if word not in stopwords.words('english')])
    words.append(review_words)

In [36]:
import keras

In [59]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense,Conv2D ,LSTM,Lambda
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras import backend as K
tf_session = K.get_session()
import tensorflow as tf

In [62]:
def stack_dim(X):
    conv_outputs=tf.unstack(X,axis=3)
    return tf.concat(conv_outputs,axis=1)

def get_shape(input_shape):
    assert len(input_shape)==4
    return (input_shape[0],input_shape[1]*input_shape[3],input_shape[2])

def model(input_shape,hidden_size,dense_layer1,dense_layer2):
    X_input=Input(input_shape)
    X=Conv2D(32, (5, 5), input_shape=input_shape, activation='relu')(X_input)
    X=Lambda(stack_dim,output_shape=get_shape)(X)
    X=LSTM(hidden_size,return_sequences=True)(X)
    X=Dense(dense_layer1, activation='relu')(X)
    X=Dense(dense_layer2, activation='relu')(X)
    model = Model(inputs = X_input, outputs = X, name='sent_classifier')
    return model

In [63]:
import numpy as np
m=model((300,150,1),128,100,2)

In [65]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 300, 150, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 296, 146, 32)      832       
_________________________________________________________________
lambda_5 (Lambda)            (None, 9472, 146)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 9472, 128)         140800    
_________________________________________________________________
dense_3 (Dense)              (None, 9472, 100)         12900     
_________________________________________________________________
dense_4 (Dense)              (None, 9472, 2)           202       
Total params: 154,734
Trainable params: 154,734
Non-trainable params: 0
_________________________________________________________________


In [52]:
from gensim.models import Word2Vec
new_model = Word2Vec.load('model.bin')